In [16]:
import pandas as pd

# importing the data csv
df = pd.read_csv("../data/ynab_export.csv")

# cleaning inflow and outflow
df['Inflow'] = df['Inflow'].replace('[$,]', '', regex=True).astype(float)
df['Outflow'] = df['Outflow'].replace('[$,]', '', regex=True).astype(float)

#combining inflow and outflow into a single amount column
df['Amount'] = df['Inflow'] - df['Outflow']

#selecting which columns to show
df = df[['Date', 'Category', 'Amount']]

#printing for verification
df.head()

,Date,Category,Amount
0,08/31/2025,Groceries,-58.38
1,08/31/2025,Dining Out,-14.77
2,08/31/2025,bills & subscriptions,-13.39
3,08/31/2025,Books,-5.26
4,08/31/2025,Groceries,-2.99


In [17]:
import numpy as np
from tabulate import tabulate

# grouping categories by their summed amount, removing 'Ready to Assign', & sorting summed values
summary = (
    df.groupby('Category', as_index=False)['Amount'].sum()
      .query("Category != 'Ready to Assign'")
      .sort_values('Amount', ignore_index=True)
)

# building the rows of the table then adding total with a seperator row
total = float(summary['Amount'].sum())
rows = summary[['Category','Amount']].to_dict('records')
rows.append({'Category': '-----', 'Amount': np.nan})
rows.append({'Category': 'TOTAL', 'Amount': total})

# putting the table together
summary_with_total = pd.DataFrame(rows)

# adding $ to all amounts
summary_with_total['Amount_Display'] = summary_with_total['Amount'].map(
    lambda x: "" if pd.isna(x) else f"-${abs(x):,.2f}" if x < 0 else f"${x:,.2f}"
)

# final print
print(tabulate(summary_with_total[['Category','Amount_Display']],
               headers='keys', tablefmt='github', showindex=False))


| Category                            | Amount_Display   |
|-------------------------------------|------------------|
| Rent & utilities                    | -$946.18         |
| Groceries                           | -$517.57         |
| bills & subscriptions               | -$428.92         |
| 📉 Sofi Personal Loan (#2)          | -$287.34         |
| sports & rec                        | -$273.64         |
| 🚗 Ally Auto Loan (#3)              | -$144.03         |
| 📉 SoFi Student Loan (top priority) | -$129.18         |
| gas & parking                       | -$123.14         |
| car maintenance                     | -$96.62          |
| Dining Out                          | -$90.04          |
| home accessories                    | -$88.41          |
| Entertainment                       | -$49.57          |
| technology purchases                | -$20.00          |
| Personal Care                       | -$15.29          |
| Financial Costs & Investing         | -$15.26          |
